## API PARA RECOMENDACIÓN DE VIDEOJUEGOS EN STEAM 

## ETL (Extraction, Transformation, Load)

* Librerias necesarias

In [1]:
import pandas as pd
import re
import json
import gzip

Al realizar la cargar normal de los archivos se presentaban errores porque estaban en formato gzip, es por eso que antes de generar codigo éstos se extrajeron en la misma carpeta para evitar futuras complicaciones y optimizar el código.

Además, dos archivos archivos tienen sólo comilla simple (') en lugar de comillas dobles ("), es decir, los archivos estan corruptos y adicionalmente anidados. Esto también genera un error al crear el dataframe así que se prefiere extraer y limpiar los archivos uno por uno para evitar problemas futuros.

-- steam_games

Con el archivo steam_games no había ningún problema así que este es el primero que se carga:

In [2]:
# Cargar el archivo JSON en un DataFrame
steam_games = pd.read_json('output_steam_games.json', lines=True)

-- user_reviews

Ahora, para cargar (o más bien crear) el archivo user_reviews es necesario hacer algunas modificaciones, y es por eso que se prefiere guardar línea por línea para poder aplicar dichos cambios. ACLARACIÓN: Se prefiere realizar el proceso de limpieza de forma manual debido a que los recursos que se tienen para manipular los datos son muy escasos.

In [72]:
jlines = []
# Abrir el archivo JSON en modo lectura
with open('australian_user_reviews.json', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Guardar las líneas del archivo JSON
for line in lines:
    jlines.append(line)

Durante el proceso de ETL se pudo evidenciar tres casos puntuales donde en "reviews" se encontró respuestas que afectaban el procedimiento de limpieza de datos de forma negativa (se comentará más adelante el por qué), y es por esto que se decide hacer una modificación manual de "{" por "(" y de "}" por ")".

In [73]:
for i in range(len(jlines)):
    jlines[i] = jlines[i].replace('{LINK REMOVED}',"(LINK REMOVED)")
    jlines[i] = jlines[i].replace('{誰是殺手}',"(誰是殺手)")
    jlines[i] = jlines[i].replace("}{@r|)c0r3",")(@r|)c0r3")

Para tener un archivo json limpio se decide hacer un diccionario con las mismas columnas originales y luego con este crear un nuevo archivo, dado que es más efectivo y seguro que reemplazar el archivo de user_reviews. Para esto hay que extraer la información de las columnas en el archivo json original; las principales columnas son user_id, user_url y reviews.

In [74]:
# Extraer la información de las tres columnas principales del archivo json original
user_id = []
user_url = []
reviews = []

for line in jlines:
    match = re.search(r"'user_id':\s+'([^']+)'[^}]*'user_url':\s+'([^']+)'[^}]*'reviews':\s+(\[.*\])", line)
    if match:
        user_id.append(match.group(1))
        user_url.append(match.group(2))
        reviews.append(match.group(3))

Seguidamente notamos que la columna reviews a su vez tiene columnas con información útil, así que extraemos los datos de la columna reviews.

    Como se puede ver en el código, el requerimiento para extraer la información de reviews es que los caracteres estén entre "{" y "}", es por esto que comentarios como "{LINK REMOVED}" afectaba negativamente a nuestro proceso de organización. Por lo tanto, al ser sólo tres casos puntuales se decide cambiar los "{ }" por "()" respectivamente rn fichas reseñas.

In [75]:
extracted_data_reviews = []

for review in reviews:
    # Encuentra los datos entre '[' y ']' en cada revisión -> Para extraer todos los caracteres de reviews
    matches = re.findall(r'\{(.*?)\}', review)
    for match in matches:
        extracted_data_reviews.append('{'+match+'}')


Una vez con los datos individuales de cada columna limpios y listos procedemos a extraer la información de todas las columnas de reviews.

    Como algunos comentarios de review presentan comillas es necesario hacer una extraccion aparte para esta columna.

In [76]:
funny = []
posted = []
last_edited = []
item_id = []
helpful = []
recommend = []

for line in extracted_data_reviews:
    match_funny = re.search(r"'funny':\s+'([^']+)'[^}]", line)
    match_posted = re.search(r"'posted':\s+'([^']+)'", line)
    match_last_edited = re.search(r"'last_edited':\s+'([^']+)'", line)
    match_item_id = re.search(r"'item_id':\s+'([^']+)'[^}]", line)
    match_helpful = re.search(r"'helpful':\s+'([^']+)'[^}]", line)
    match_recommend = re.search(r"'recommend':\s+([a-zA-Z]+)", line)
    if match_funny:
        funny.append(match_funny.group(1))
    if not match_funny:
        funny.append("")
    if match_posted:
        posted.append(match_posted.group(1))
    if match_last_edited:
        last_edited.append(match_last_edited.group(1))
    if not match_last_edited:
        last_edited.append("")
    if match_item_id:
        item_id.append(match_item_id.group(1))
    if match_helpful:
        helpful.append(match_helpful.group(1))
    if match_recommend:
        recommend.append(match_recommend.group(1))

In [77]:
review = []
for i in range(len(extracted_data_reviews)):
    data = extracted_data_reviews[i]

    # Encontrar la posición del inicio de la reseña
    review_start = data.find("'review': ") + len("'review': ")

    # Encontrar la posición del final de la reseña
    review_end = data.find("}", review_start)

    # Extraer el texto de la reseña
    review_text = data[review_start:review_end]

    # Reemplazar comillas dobles escapadas con comillas dobles normales
    review_text = review_text.replace('\'\'', '\'').replace('"',"'")
    review.append(review_text)

for i in range(len(review)):
    if review[i].startswith("'") and review[i].endswith("'"):
        review[i] = review[i][1:-1]

Finalmente tenemos todos los datos limpios, y si no se ha notado, una vez que éstos se extraen se les agrega las comillas dobles (") de manera automática, entonces se ha cumplido el objetivo principal de la transformación del archivo user_reviews. Sin embargo, también se nota que dentro de reviews hay varios comentarios, por ende "funny", que es la primera columna de reviews se repite varias veces; para asegurarnos que todo esté bien se hace una lista que contenga la cantidad de veces que aparezca "funny" en cada fila de reviews.

In [83]:
len(reviews)

25799

In [84]:
len(review)

59305

In [78]:
counter = []
for i in range(len(reviews)):
    count_funny = reviews[i].count("'funny':")
    counter.append(count_funny)

Ahora comprobamos que la suma de nuestro contador y que la cantidad de datos de funny sea la misma. Como todo está en orden entonces estamos listos para formar nuestro diccionario, pero nótese que se deben formar dos diccionarios: Uno para reviews y otro para data_reviews en general, dado que reviews en sí tiene columnas que organizar.

In [146]:
len(funny) - sum(counter)

0

Cada fila de nuestro archivo final debe tener aproximadamente la siguiente estructura:

    data_reviews = {
        "user_id" : user_id[i],
        "user_url" : user_url[i],
        "reviews" : reviews_modified[i]
    }

Primero organizamos el diccionario para reviews:

Se crea la función form_review que se encarga de crear el diccionario linea por linea para cada review. Finalmente se recopilan los reviews en un diccionario final.

In [79]:
def form_review(count):
    t = sum(counter[:counter[count]])
    reviews_line = []
    for i in range(sum(counter[:count]), sum(counter[:count])  + counter[count]):
        new_review = {
            "funny" : funny[i],
            "posted" : posted[i],
            "last_edited" : last_edited[i],
            "item_id" : item_id[i],
            "helpful" : helpful[i],
            "recommend" : recommend[i],
            "review" : review[i]
        }
        reviews_line.append(new_review)
    return reviews_line

In [80]:
reviews_modified = []
for i in range(len(counter)):
    reviews_modified.append(form_review(i))

Ahora se crea el diccionario final para nuestro archivo json ...

In [81]:
data_reviews = []
for i in range(len(user_id)):
    data = {
        "user_id" : user_id[i],
        "user_url" : user_url[i],
        "reviews" : reviews_modified[i]
    }  
    data_reviews.append(data)

In [82]:
# Guardar los datos en un archivo JSON
with open('data_reviews.json', 'w') as json_file:
    for review_data in data_reviews:
        json.dump(review_data, json_file)
        json_file.write('\n')

... y finalmente cargamos esto en nuestro DataFrame user_reviews

In [83]:
# Cargar el archivo JSON en un DataFrame
user_reviews = pd.read_json('data_reviews.json', lines=True)

-- users_items

Ahora, necesitamos hacer el mismo proceso de limpieza pero para crear el archivo users_items. Se sigue un proceso muy similar que con user_reviews, por no decir que es prácticamente el mismo. 

    No se presentaron cambios significativos en el código, así que no se comentará este extracto por extracto.

In [19]:
jlines = []
# Abrir el archivo JSON en modo lectura
with open('australian_users_items.json', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Guardar las líneas del archivo JSON
for line in lines:
    jlines.append(line)

In [20]:
user_id = []
items_count = []
steam_id = []
user_url = []

# Patrón de búsqueda usando comillas simples en lugar de dobles
pattern = r"'user_id':\s+'([^']+)'[^}]*'items_count':\s+(\d+)[^}]*'steam_id':\s+'([^']+)'[^}]*'user_url':\s+'([^']+)'"

for i in range(len(jlines)):
    # Contenido de la línea con comillas simples
    line = jlines[i]

    match = re.search(pattern, line)
    if match:
        user_id.append(match.group(1))
        items_count.append(int(match.group(2)))
        steam_id.append(match.group(3))
        user_url.append(match.group(4))

In [21]:
items = []

for i in range(len(jlines)):
    # Contenido de la línea con comillas simples
    line = jlines[i]

    # Encontrar la posición del inicio de la lista de items
    items_start = line.find("\'items\': ") + len("\'items\': ")

    # Extraer la lista de items (incluyendo los corchetes)
    items_info = line[items_start:]

    # Eliminar el corchete final si está presente
    if items_info.endswith("}"):
        items_info = items_info[:-1]
    
    items.append(items_info)

In [22]:
extracted_data_items = []

for item in items:
    # Encuentra los datos entre '[' y ']' en cada revisión -> Para extraer todos los caracteres de reviews
    matches = re.findall(r'\{(.*?)\}', item)
    for match in matches:
        extracted_data_items.append('{'+match+'}')

In [24]:
item_id = []
item_name = []
playtime_forever = []
playtime_2weeks = []

# Línea de ejemplo
for i in range(len(extracted_data_items)):
    line = extracted_data_items[i]
    match_item_id = re.search(r"'item_id':\s+'([^']+)'[^}]", line)
    match_item_name = re.search(r"'item_name':\s+\"([^\"]+)\"", line)
    match_playtime_forever = re.search(r"'playtime_forever':\s+(\d+)[^}]", line)
    match_playtime_2weeks = re.search(r"'playtime_2weeks':\s+(\d+)", line)

    if match_item_id:
        item_id.append(match_item_id.group(1))
    if match_item_name:
        item_name.append(match_item_name.group(1))
    if not match_item_name:
        match_item_name = re.search(r"'item_name':\s+'([^']+)'", line)
        if match_item_name:
            item_name.append(match_item_name.group(1))
    if match_playtime_forever:
        playtime_forever.append(match_playtime_forever.group(1))
    if match_playtime_2weeks:
        playtime_2weeks.append(match_playtime_2weeks.group(1))


In [25]:
counter = []
for i in range(len(items)):
    count_item_id = items[i].count("'item_id':")
    counter.append(count_item_id)

In [26]:
def form_item(count):
    t = sum(counter[:counter[count]])
    item_line = []
    for i in range(sum(counter[:count]), sum(counter[:count])  + counter[count]):
        new_item = {
            "item_id" : item_id[i],
            "item_name" : item_name[i],
            "playtime_forever" : playtime_forever[i],
            "playtime_2weeks" : playtime_2weeks[i]
        }
        item_line.append(new_item)
    return item_line

In [27]:
items_modified = []
for i in range(len(counter)):
    items_modified.append(form_item(i))

In [28]:
data_items = []
for i in range(len(user_id)):
    data = {
        "user_id" : user_id[i],
        "items_count" : items_count[i],
        "steam_id" : steam_id[i],
        "user_url" : user_url[i],
        "items" : items_modified[i]
    }  
    data_items.append(data)

In [29]:
# Guardar los datos en un archivo JSON
with open('data_items.json', 'w') as json_file:
    for items_data in data_items:
        json.dump(items_data, json_file)
        json_file.write('\n')

In [30]:
# Cargar el archivo JSON en un DataFrame
users_items = pd.read_json('data_items.json', lines=True)

## NLP (Natural Language Processing)

Ahora, ya se tienen los datos limpios pero al ser tantos y tener recursos tan limitados lo mejor sería encontrar una forma de optimizar dichos datos. Se ve potencial en reemplazar las reseñas de user_reviews['reviews']..['reviews'] por un análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. 

* Librerias necesarias 

In [32]:
from langdetect import detect
import re
from deep_translator import GoogleTranslator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

Para hacer el NLP el primer obstáculo que presentamos es que algunos de las reseñas se encuentran en otros idiomas diferentes del inglés (que es el idioma en que tenemos los datos), así que a modo de exploración nos conviene saber la cantidad de frases que no se encuentran en inglés.

In [33]:
language = []
index = []
phrases = []
pattern = re.compile(r'^[a-zA-Z]+$')

for i, text in enumerate(review):
    if bool(pattern.match(text)):
        detected_language = detect(text)
        if detected_language != 'en':
            language.append(detected_language) 
            index.append(i) # Es una excelente idea saber en qué indice dento de reviews se encuentra ubicada la reseña
            phrases.append(review[i])


Dado que tenemos un número considerable de reseñas en un idioma diferente al inglés, estaría bien no perder esos datos sino más bien traducir dichas reseñas al inglés y luego reemplzar dichos comentarios.

    Si bien muchas frases identificadas en language como que pertenecen a otro idioma en realidad no es así, esto no afecta en nada porque finalmente se van a traducir al inglés.

In [34]:
translated = []
for i in range(len(phrases)):
    if len(phrases[i]) < 5000:    # Hay un comentario no útil que tienen más de 5000 caracteres
        traductor = GoogleTranslator(source=language[i], target='en')
        translated.append(traductor.translate(phrases[i]))
    else:
        translated.append(phrases[i])

In [84]:
# Reemplazar las traducciones de las reseñas
for i in range(len(translated)):
    review[index[i]] = translated[i]

Ahora que se tiene todas las reseñas en inglés se vuelve a formar el archivo json pero con la información modificada.

In [85]:
counter = []
for i in range(len(reviews)):
    count_funny = reviews[i].count("'funny':")
    counter.append(count_funny)

In [86]:
reviews_modified = []
for i in range(len(counter)):
    reviews_modified.append(form_review(i))

In [87]:
# Extraer la información de las tres columnas principales del archivo json original
user_id = []
user_url = []
reviews = []

for line in jlines:
    match = re.search(r"'user_id':\s+'([^']+)'[^}]*'user_url':\s+'([^']+)'[^}]*'reviews':\s+(\[.*\])", line)
    if match:
        user_id.append(match.group(1))
        user_url.append(match.group(2))
        reviews.append(match.group(3))

In [88]:
data_reviews = []
for i in range(len(user_id)):
    data = {
        "user_id" : user_id[i],
        "user_url" : user_url[i],
        "reviews" : reviews_modified[i]
    }  
    data_reviews.append(data)

In [89]:
# Guardar los datos en un archivo JSON
with open('data_reviews.json', 'w') as json_file:
    for review_data in data_reviews:
        json.dump(review_data, json_file)
        json_file.write('\n')

In [90]:
# Guardar el DataFrame en un archivo JSON
user_reviews.to_json('data_reviews.json', orient='records', lines=True)

Ya que se tienen todas las reseñas en el idioma adecuado, ahora se hace el modelaje de NLP. Se crean las variables y se entrena el modelo, como lo que se quiere es clasificar las reseñas como positivas, negativas o neutrales, un clasificador de Naive Bayes Gaussiano sería el preciso para ésta tarea.

In [91]:
df_reviews = pd.DataFrame(review) # Se crea el DataFrame de reviews para que se puedan procesar todos los datos
cv = CountVectorizer(max_features = 1500)# solo toma 1500 palabras
X = cv.fit_transform(df_reviews[0]).toarray() 
y = df_reviews.iloc[:, -1].values

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.02, random_state = 0)

In [53]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

A continuación se procede con realizar un análisis de sentimiento en cada reseña. Primero se inicializan las herramientas sia, ps y all_stopwords; luego se crea la función analyze_review y finalmente se realiza la clasificación.

In [54]:
# Inicializar el analizador de sentimientos
sia = SentimentIntensityAnalyzer() 

# Inicializar el stemmer
ps = PorterStemmer()

# Obtener la lista de stopwords en inglés
all_stopwords = set(stopwords.words('english'))

def analyze_review(new_review):
    # Preprocesamiento del texto
    new_review = re.sub('[^a-zA-Z]', ' ', new_review.lower())
    new_review = new_review.replace(" t ", "ot ")
    new_review = ' '.join([ps.stem(word) for word in new_review.split() if word not in all_stopwords])

    # Convertir el texto preprocesado en matriz
    new_corpus = [new_review]
    new_X_test = cv.transform(new_corpus).toarray()

    # Calcular el puntaje de sentimiento
    sentiment_score = sia.polarity_scores(new_review)['compound']

    # Determinar el sentimiento basado en el puntaje
    if sentiment_score > 0:
        sentiment = 2  # Positivo
    elif sentiment_score < 0:
        sentiment = 0  # Negativo
    else:
        sentiment = 1  # Neutral"""
    return sentiment

In [93]:
classification = []
for i in range(len(review)):
    classification.append(analyze_review(review[i]))

Ahora que tenemos la clasificación de cada reseña, se reemplaza dicha reseña por su calificación. Seguidamente se guardan los datos en el archivo json.

In [94]:
# Crear un diccionario que mapea las revisiones a sus clasificaciones correspondientes
mapeo_revisiones = {review[i]: classification[i] for i in range(len(review))}

# Recorrer el DataFrame user_reviews_copy una sola vez
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        review_actual = user_reviews['reviews'][i][j]['review']
        # Verificar si la revisión actual está en el diccionario de mapeo
        if review_actual in mapeo_revisiones:
            # Reemplazar la revisión con su clasificación correspondiente
            user_reviews['reviews'][i][j]['review'] = mapeo_revisiones[review_actual]


In [95]:
# Guardar los datos en un archivo JSON
user_reviews.to_json('data_reviews.json', orient='records', lines=True)

## Funciones alimentadoras de la API

Ahora se desea hacer una API para facilitar algunas consultas en específico. 

* Librerias necesarias

In [16]:
from datetime import datetime

La primera función es def userdata( User_id : str ): La cual devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [12]:
def userdata( User_id : str ): 
    users = list(set(users_items['user_id']))
    if User_id in users:
        # Filtrar los juegos comprados por el usuario
        user_games = set()  # Evita tener datos duplicados
        for _, row in users_items[users_items['user_id'] == User_id].iterrows():
            user_games.update(item['item_name'] for item in row['items'])
        
        # Calcular el dinero gastado por el usuario
        amount_spend = 0
        for _, game_row in steam_games[steam_games['app_name'].isin(user_games)].iterrows():
            game_price = game_row['price']
            if game_price not in ['Free to Play', 'Free', None, 'Third-party','Free To Play','Free Movie','Install Now']:
                amount_spend += float(game_price)

        # Calcular la cantidad de juegos que tiene el usuario
        cant_items = len(user_games)

        # Calcular el porcentaje de recomendación
        total = 0
        recomendados = 0
        for _, row in user_reviews[user_reviews['user_id'] == User_id].iterrows():
            for review in row['reviews']:
                if review['recommend'] == 'True':
                    recomendados += 1
                total += 1
        porcentaje_recom = (recomendados / total) * 100 

        print(f'La cantidad de dinero gastado por el usuario es: ${round(amount_spend, 2)}\n'
            f'Porcentaje de recomendación de juegos: {porcentaje_recom:.2f}%\n'
            f'Cantidad de juegos que tiene el usuario: {cant_items}')
    else:
        print('El usuario que brinda no se encuentra en la base de datos.')

In [13]:
userdata('js41637')

La cantidad de dinero gastado por el usuario es: $6165.06
Porcentaje de recomendación de juegos: 100.00%
Cantidad de juegos que tiene el usuario: 885


Ahora se hace la función def countreviews(first_date, last_date : str ): La cual retorna la cantidad de usuarios que realizaron reviews entre dos fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend. Primero se hace un ajuste en el formato de las fechas, luego se ejecuta la función is_valid_date para identificar si las fechas están completas, y finalmente se crea la funcion countreviews.

In [139]:
# Primero se deben modificar las fechas que se tienen en el archivo json
def update_date(original_date):
    try:
        parsed_date = datetime.strptime(original_date, "%B %d %Y")
        formatted_date = parsed_date.strftime("%Y-%m-%d")
        return formatted_date
    except ValueError:
        pass

In [140]:
# Se aplica la función update_date para reemplazar las fechas en user_reviews
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        if user_reviews['reviews'][i][j]['posted'] == None:
            pass
        else:
            user_reviews['reviews'][i][j]['posted'] = update_date(user_reviews['reviews'][i][j]['posted'].replace(',','').replace('.','').replace('Posted ',''))

In [141]:
# Guardar los datos modificados en un archivo JSON
user_reviews.to_json('data_reviews.json', orient='records', lines=True)

In [18]:
def is_valid_date(date_string):
    try:
        datetime.strptime(date_string, '%Y-%m-%d')
        return True
    except ValueError:
        return False

    También se debe saber cuál es la primera y última fecha en reviews para que la función retorne valores dentro de ese mismo rango de fechas

In [143]:
dates = [review['posted'] for reviews_list in user_reviews['reviews'] for review in reviews_list]
fechas_validas = [fecha for fecha in dates if fecha is not None]
fechas_datetime = [datetime.strptime(fecha, '%Y-%m-%d') for fecha in fechas_validas]
print(f'Primera fecha:{min(fechas_datetime)}')
print(f'última fecha:{max(fechas_datetime)}')

Primera fecha:2010-10-16 00:00:00
última fecha:2015-12-31 00:00:00


In [14]:
def countreviews(first_date, last_date : str ): 
    if datetime.strptime(first_date, '%Y-%m-%d') > datetime.strptime('2010-10-16', '%Y-%m-%d') and datetime.strptime(last_date, '%Y-%m-%d') < datetime.strptime('2015-12-31', '%Y-%m-%d') :
        usuarios = set()
        total = 0
        recomendados = 0
        
        for i in range(len(user_reviews)):
            for review in user_reviews['reviews'][i]:
                posted_date = review['posted']
                if posted_date and is_valid_date(posted_date):
                    if first_date <= posted_date <= last_date:
                        usuarios.add(user_reviews['user_id'][i])
                        if review['recommend'] == "True":
                            total += 1
                            recomendados += 1
                        else:
                            total += 1
        
        usuarios_count = len(usuarios)
        porcentaje_recom = (recomendados / total) * 100
        
        print(f'Cantidad de usuarios que realizaron reviews entre las fechas {first_date} y {last_date} es de: {usuarios_count}\n'
            f'Porcentaje de juegos recomendados entre las fechas {first_date} y {last_date} es de: {round(porcentaje_recom, 2)}%')
    else:
        print('El rango de fechas dado está por fuera del de la base de datos. Se encuentra información desde 2010-10-16 hasta 2015-12-31')


In [21]:
countreviews('2012-01-01','2015-01-01')

Cantidad de usuarios que realizaron reviews entre las fechas 2012-01-01 y 2015-01-01 es de: 16548
Porcentaje de juegos recomendados entre las fechas 2012-01-01 y 2015-01-01 es de: 92.31%


Ahora se crea la función def genre( género : str ): Que devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

Primero se crean las funciones playtime(para extraer el total de playtime_forever por cada juego), game_genres(retorna los género de un juego dado), genres_list(retorna una lista con el total de géneros); para finalmente hacer la función genre.

In [22]:
def playtime(game):
    filtered_items = [item['playtime_forever'] for items in users_items['items'] for item in items if item['item_name'] == game]
    total_playtime = sum(map(int, filtered_items))
    return total_playtime

In [23]:
def game_genres(game):
    matching_games = steam_games[steam_games['app_name'] == game]
    genres = [genre for genres_list in matching_games['genres'].dropna() for genre in genres_list]
    return genres

In [24]:
def genres_list():
    # Descomponer las listas de géneros en filas separadas
    exploded_genres = steam_games['genres'].explode()
    # Obtener los géneros únicos y eliminar los valores nulos
    unique_genres = exploded_genres.dropna().unique()
    return unique_genres

In [25]:
def genre( género : str ):
    g_list = genres_list()
    if género in g_list:
        games = set(steam_games['app_name'])  # Utilizar un conjunto en lugar de una lista para buscar de manera más eficiente
        total_playtime = {}

        for i in range(len(users_items['items'])):
            for item in users_items['items'][i]:
                if 'playtime_forever' in item:
                    game_name = item['item_name']
                    playtime_forever = int(item['playtime_forever'])
                    
                    if game_name in games:
                        if game_name not in total_playtime:
                            total_playtime[game_name] = playtime_forever
                        else:
                            total_playtime[game_name] += playtime_forever

        games_by_genre = {}

        for i in range(len(steam_games)):
            app_name = steam_games['app_name'][i]
            genres = steam_games['genres'][i]
            
            if genres is not None:
                for genre in genres:
                    if genre not in games_by_genre:
                        games_by_genre[genre] = []
                    games_by_genre[genre].append(app_name)

        total_playtime_by_genre = {}  # Diccionario para almacenar el tiempo total de juego por género

        for i in range(len(g_list)):
            genre = g_list[i]
            if genre in games_by_genre:
                total_time = 0  # Inicializar el tiempo total en cero para este género
                for game in games_by_genre[genre]:
                    if game in total_playtime:
                        total_time += total_playtime[game]
                total_playtime_by_genre[genre] = total_time

        sorted_genres_by_playtime = sorted(total_playtime_by_genre.items(), key=lambda x: x[1], reverse=True)
        
        for i in range(len(sorted_genres_by_playtime)):
            if sorted_genres_by_playtime[i][0] == género:
                print(f'El género {género} se encuentra en el puesto número {i+1} sobre el ranking de los géneros con mayor cantidad de tiempo jugado')
    
    else:
        print('El género brindado no se encuentra en la base de datos, por favor revíselo')

In [27]:
genre('Simulation')

El género Simulation se encuentra en el puesto número 4 sobre el ranking de los géneros con mayor cantidad de tiempo jugado


También se hace la función def userforgenre( género : str ): Que retorna el top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [28]:
def userforgenre(género: str):
    g_list = genres_list()
    if género in g_list:
        user_games_playtime = {}
        games_by_genre = {}

        for i in range(len(users_items['user_id'])):
            user_id = users_items['user_id'][i]
            for item in users_items['items'][i]:
                if 'playtime_forever' in item:
                    game_name = item['item_name']
                    playtime_forever = int(item['playtime_forever'])
                    
                    if user_id not in user_games_playtime:
                        user_games_playtime[user_id] = []
                    user_games_playtime[user_id].append((game_name, playtime_forever))

        for i in range(len(steam_games)):
            app_name = steam_games['app_name'][i]
            genres = steam_games['genres'][i]
                
            if genres is not None:
                for genre in genres:
                    if genre not in games_by_genre:
                        games_by_genre[genre] = []
                    games_by_genre[genre].append(app_name)

        target_genre_games = set(games_by_genre.get(género, []))

        genre_user_playtime = []
        for user_id, games_playtime in user_games_playtime.items():
            user_genre_playtime = 0
            for game_name, playtime_forever in games_playtime:
                if game_name in target_genre_games:
                    user_genre_playtime += playtime_forever
            genre_user_playtime.append((user_id, user_genre_playtime))
        
        # Ordenar la lista en función del tiempo de juego y obtener los 5 usuarios con más playtime
        best_gamers = sorted(genre_user_playtime, key=lambda x: x[1], reverse=True)[:5]

        user_ids = [user_id for user_id, _ in best_gamers]
        user_urls = [users_items['user_url'][users_items['user_id'] == user_id].iloc[0] for user_id in user_ids]

        result_dict = {'user_id': user_ids, 'user_url': user_urls}
        result_df = pd.DataFrame(result_dict)
        result_df.index += 1 # Para que el índice inicie en 1

        return result_df
    else:
        print('El género brindado no se encuentra en la base de datos, por favor revíselo')

In [30]:
userforgenre('Simulation')

,user_id,user_url
1,jimmynoe,http://steamcommunity.com/id/jimmynoe
2,clawbot44,http://steamcommunity.com/id/clawbot44
3,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T
4,Evilutional,http://steamcommunity.com/id/Evilutional
5,tsunamitad,http://steamcommunity.com/id/tsunamitad


Otra de las funciones es def developer( desarrollador : str ): Que debe retornar la cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [189]:
# Primero se debe saber que tipo de contenido free hay
prices = list(set(steam_games['price']))
posibles = []
for i in range(len(prices)):
    if type(prices[i]) != float:
        posibles.append(prices[i])
posibles = list(set(posibles))
print(posibles)

['Free to Try', 'Play Now', 'Play for Free!', 'Play WARMACHINE: Tactics Demo', 'Free to Play', 'Play the Demo', 'Install Theme', None, 'Third-party', 'Free', 'Free HITMAN™ Holiday Pack', 'Starting at $449.00', 'Install Now', 'Free Demo', 'Starting at $499.00', 'Free Mod', 'Free Movie', 'Free to Use', 'Free To Play']


In [31]:
def developer(desarrollador: str):
    """Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:"""
    developers = list(set(steam_games['developer']))
    if desarrollador in developers:
        años = []
        cantidad = 0
        free = 0
        
        for i in range(len(steam_games)):
            if steam_games['developer'][i] == desarrollador:
                cantidad += 1
                if steam_games['price'][i] in ['Free Demo', 'Free Mod', 'Free to Use', 'Free To Play', 'Free Movie', 'Play for Free!', 'Free to Try', 'Free', 'Free to Play', 'Free HITMAN™ Holiday Pack']:
                    free += 1
                if steam_games['release_date'][i] is not None and is_valid_date(steam_games['release_date'][i]):
                    año = steam_games['release_date'][i][:4]
                    años.append(año)
        años = list(set(años))  # Eliminar duplicados
        porcentaje_free_por_año = (free/cantidad)*100

        print(f'Cantidad total de juegos: {cantidad}')
        if cantidad != 0:
            print(f'Porcentaje de contenido free: {porcentaje_free_por_año:.2f}%')

        df = pd.DataFrame()
        df[''] = años
        df[desarrollador] = (str(round(porcentaje_free_por_año,2)))+'%'
        return df
    else: 
        print('El desarrollador brindado no se encuentra en la base de datos, por favor revíselo')

In [34]:
developer('ExtinctionArts')

Cantidad total de juegos: 1
Porcentaje de contenido free: 0.00%


,,ExtinctionArts
0,2017,0.0%


Finalmente, se hace la funcion def sentiment_analysis( año : int ): Que según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

In [35]:
def sentiment_analysis( año : int ): 
    if año > 2009 and año < 2016:
        positivo = 0
        negativo = 0
        neutral = 0
        for i in range(len(user_reviews)):
            for j in range(len(user_reviews['reviews'][i])):
                if user_reviews['reviews'][i][j]['posted'] is None:
                    pass
                if user_reviews['reviews'][i][j]['posted'] is not None:
                    if int(user_reviews['reviews'][i][j]['posted'][:4]) == año:
                        if user_reviews['reviews'][i][j]['review'] == 2:
                            positivo += 1
                        elif user_reviews['reviews'][i][j]['review'] == 1:
                            neutral += 1
                        else:
                            negativo += 1

        resultado = {'Positivo':positivo, 'Neutral':neutral,'Negativo':negativo}
        return resultado
    else:
        print('El añodado está por fuera del de la base de datos. Se encuentra información desde 2010-10-16 hasta 2015-12-31')

In [36]:
sentiment_analysis(2013)

{'Positivo': 4310, 'Neutral': 1698, 'Negativo': 784}

## Construcción de la API

* Primero se crea el entorno virtual con el comando python -m venv proyecto-env

* Se activa el entorno virtual: proyecto-env\Scripts\activate.bat

* Se instala fastapi: pip install fastapi

* Instalar uvicorn: pip install "uvicorn[standard]"

* Hacer el freeze de los requirements: pip freeze > requirements.txt --> Si luego se necesita instalar otra librería más, se vuelve a ejecutar este comando.

* Se crea el archivo main.py y se importa FastAPI: 

    from fastapi import FastAPI

    app = FastAPI()

* Se forman todas las funciones necesarias

* Levantar el servidor: python -m uvicorn main:app --reload

Ahora que tenemos todas las funciones para nuestra Api, es útil modificar los archivos y eliminar todas las columnas que no se utilizan para garantizar que el espacio y el rendimiento.

In [ ]:
del steam_games['publisher']
del steam_games['title']
del steam_games['url']
del steam_games['tags']
del steam_games['specs']
del steam_games['early_access']
del steam_games['id']

In [181]:
# Identifica las filas donde todas las columnas son None
rows_to_remove = steam_games[steam_games.isnull().all(axis=1)].index

# Luego, utiliza el método 'drop' para eliminar estas filas del DataFrame
steam_games = steam_games.drop(rows_to_remove)

# Resetear los index
steam_games.reset_index(drop=True, inplace=True)

In [182]:
steam_games.to_json('output_steam_games.json', orient='records', lines=True)

In [ ]:
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        del user_reviews['reviews'][i][j]['funny']

In [ ]:
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        del user_reviews['reviews'][i][j]['last_edited']

In [ ]:
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        del user_reviews['reviews'][i][j]['item_id']

In [ ]:
for i in range(len(user_reviews)):
    for j in range(len(user_reviews['reviews'][i])):
        del user_reviews['reviews'][i][j]['helpful']

In [ ]:
user_reviews.to_json('data_reviews.json', orient='records', lines=True)

In [ ]:
del users_items['items_count']
del users_items['steam_id']

In [ ]:
for i in range(len(users_items)):
    for j in range(len(users_items['items'][i])):
        del users_items['items'][i][j]['item_id']

In [ ]:
for i in range(len(users_items)):
    for j in range(len(users_items['items'][i])):
        del users_items['items'][i][j]['playtime_2weeks']

In [ ]:
users_items.to_json('data_items.json', orient='records', lines=True)

    El archivo de data_items sigue pesando mucho, es por eso que se va a comprimir y luego leer el dataframe de nuevo para asegurar que todo está bien

In [208]:
# Para comprimir el archivo
with open('data_items.json', 'rb') as archivo_json:
    with gzip.open('data_items.json' + '.gz', 'wb') as archivo_json_comprimido:
        archivo_json_comprimido.writelines(archivo_json)

In [234]:
# Para leer el archivo json.gzip
with gzip.open('data_items.json.gz', 'rb') as archivo_json_comprimido:
    users_items = pd.read_json(archivo_json_comprimido, lines=True, orient='records')

-----------------------------------------------------------------------------------------------------------------------------------------------------------

In [9]:
users = []
with gzip.open('data_items.json.gz', 'rb') as archivo_json_comprimido:
    for linea in archivo_json_comprimido:
        datos_json = linea.decode('utf-8')
        objeto_json = json.loads(datos_json)
        # Añade el objeto JSON completo a la lista
        users.append(objeto_json)
# Crea un DataFrame a partir de la lista de objetos JSON
users_items = pd.DataFrame(users)
steam_games = pd.read_json('output_steam_games.json', lines=True)
user_reviews = pd.read_json('data_reviews.json', lines=True)